In [11]:
import spacy
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import nltk
import re,string
nltk.download('stopwords')
from nltk.corpus import stopwords
stopword=set(stopwords.words('english'))
stemmer = nltk.SnowballStemmer("english")

#read the dataset with name "Fake_Real_Data.csv" and store it in a variable df
df = pd.read_csv("C:\\Users\\Admin\\Downloads\\twitter_data.csv")

#print the shape of dataframe
print(df.shape)

#print top 5 rows
df.head(5)

(24783, 7)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [12]:
nlp = spacy.load("en_core_web_lg")

In [13]:
df["labels"] = df["class"].map({0: "Hate Speech", 1: "Offensive Speech", 2: "No Hate and Offensive Speech"})
df = df[["tweet", "labels"]]

In [9]:
df["labels"] = df["class"].map({0: "No Hate and Offensive Speech", 1: "Offensive Speech"})
df = df[["tweet", "labels"]]

KeyError: 'class'

In [14]:
 def preprocess(text):
        text = str(text)
        test_list = text.split()
        text = ""
        for word in test_list:
            if word.endswith("ing"):
                word=word[:-3]
            text+=word
            text+=" "
        doc = nlp(text)
        filtered_tokens = []
        
        for token in doc:
            if token.is_stop or token.is_punct:
                continue
            filtered_tokens.append(token.lemma_)
            
        return " ".join(filtered_tokens)

In [15]:
df["tweet_new"] = df.tweet.apply(preprocess)

In [16]:
df.head()

,tweet,labels,tweet_new
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive Speech,RT @mayasolovely woman complain clean house am...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Speech,RT @mleew17 boy dats cold tyga dwn bad cuffin ...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Speech,RT @urkindofbrand Dawg RT @80sbaby4life fuck b...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Speech,RT @C_G_Anderson @viva_based look like tranny
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Speech,RT @ShenikaRoberts shit hear true faker bitch ...


In [17]:
def clean(text):
    text = str (text). lower()
    text = re. sub('[.?]', '', text)
    text = re. sub('https?://\S+|www.\S+', '', text)
    text = re. sub('<.?>+', '', text)
    text = re. sub('[%s]' % re. escape(string. punctuation), '', text)
    text = re. sub('\n', '', text)
    text = re. sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ". join(text)
    text = [stemmer. stem(word) for word in text. split(' ')]
    text=" ". join(text)
    return text

In [18]:
df["tweet_new"] = df.tweet_new.apply(clean)

In [19]:
x = np. array(df["tweet_new"])
y = np. array(df["labels"])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [20]:
# Load a pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [21]:
# Define a function to convert a sentence to BERT embeddings using Hugging Face Transformers
def sentence_to_bert_embeddings(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze(0).numpy()

In [22]:
# Convert training data to BERT embeddings
X_train_bert = [sentence_to_bert_embeddings(sentence, tokenizer, model) for sentence in X_train]

In [23]:
# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Split data into training and testing
X_train_bert, X_test_bert, y_train_enc, y_test_enc = train_test_split(X_train_bert, y_train_encoded, test_size=0.2, random_state=42)

In [24]:
# Define and train an SVM classifier
clf = SVC()
clf.fit(X_train_bert, y_train_enc)

#Testing the model
y_pred = clf.predict (X_test_bert)

#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test_enc,y_pred))

# Preprocess and clean the new input
new_string = "hello everyone"
new_string = preprocess(new_string)

# Convert the new input to BERT embeddings
new_string_embedding = sentence_to_bert_embeddings(new_string, tokenizer, model).reshape(1, -1)

# Predict the label
predicted_label_encoded = clf.predict(new_string_embedding)
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)

print("Predicted Label:", predicted_label)

0.8638964167419452
Predicted Label: ['No Hate and Offensive Speech']


In [25]:
from sklearn.tree import DecisionTreeClassifier

#Model building
model1 = DecisionTreeClassifier()
#Training the model
model1.fit(X_train_bert, y_train_enc)
#Testing the model
y_pred = model1.predict (X_test_bert)
#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test_enc,y_pred))

# Preprocess and clean the new input
new_string = "get the hell out of here"
new_string = preprocess(new_string)

# Convert the new input to BERT embeddings
new_string_embedding = sentence_to_bert_embeddings(new_string, tokenizer, model).reshape(1, -1)

# Predict the label
predicted_label_encoded = model1.predict(new_string_embedding)
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)

print("Predicted Label:", predicted_label)

0.7308039747064138
Predicted Label: ['Offensive Speech']


In [26]:
from sklearn.ensemble import RandomForestClassifier

#Model building
model1 = RandomForestClassifier()
#Training the model
model1.fit(X_train_bert,y_train_enc)
#Testing the model
y_pred = model1.predict (X_test_bert)
#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test_enc,y_pred))

# Preprocess and clean the new input
new_string = "Nice! Let's do it"
new_string = preprocess(new_string)

# Convert the new input to BERT embeddings
new_string_embedding = sentence_to_bert_embeddings(new_string, tokenizer, model).reshape(1, -1)

# Predict the label
predicted_label_encoded = model1.predict(new_string_embedding)
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)

print("Predicted Label:", predicted_label)

0.8292682926829268
Predicted Label: ['Offensive Speech']


In [27]:
from sklearn.ensemble import VotingClassifier

# Initialize individual models
dt_model = DecisionTreeClassifier()
rf_model = RandomForestClassifier()
svm_model = SVC(kernel='linear', probability=True)  # Enable probability estimation for SVM

# Define the ensemble of models
ensemble_model = VotingClassifier(estimators=[
    ('decision_tree', dt_model),
    ('random_forest', rf_model),
    ('svm', svm_model)
], voting='soft')  # Use soft voting for probability averaging

# Training the ensemble model
ensemble_model.fit(X_train_bert, y_train_enc)

# Testing the ensemble model
y_pred_ensemble = ensemble_model.predict(X_test_bert)

# Accuracy Score of ensemble model
print("Ensemble Accuracy Score:", accuracy_score(y_test_enc, y_pred_ensemble))

Ensemble Accuracy Score: 0.8196326407708522


In [32]:
# Get input from the user
user_input = input("Enter the text you want to classify: ")

# Preprocess the user input
user_input_processed = preprocess(user_input)

# Transform the preprocessed input using the TF-IDF vectorizer
user_input_vectorized = sentence_to_bert_embeddings(user_input_processed, tokenizer, model).reshape(1, -1)

# Make prediction using the ensemble model
prediction = ensemble_model.predict(user_input_vectorized)

# Print the predicted label
print("Predicted label:", prediction)

Enter the text you want to classify: hello
Predicted label: [2]
